In [291]:
import pandas as pd
import altair as alt
import warnings
import numpy as np
import requests
import datetime
warnings.filterwarnings("ignore")
alt.renderers.set_embed_options(theme='dark')
pd.set_option("display.max_colwidth", 400)
pd.set_option("display.max_rows", 400)


In [292]:
class AMPSDataProvider:
    
    def __init__(self):
        pass
        
    def load(self):
        today = datetime.datetime.today().date()
        df = None
        for i in range(100):
            date = today-datetime.timedelta(days=int(i))
            date = date.strftime("%Y%m%d")
            file_name = f'../data/amps_{date}_1.csv'
            try:
                df = pd.read_csv(file_name, index_col=0)
                print(f'File found: {file_name}')
                break
            except:
                print(f'File not found: {file_name}')
        self.amps = df
        
    def parse(self):
        df = self.amps
        df['boost_accrual_start_date'] = pd.to_datetime(df['boost_accrual_start_time'], unit='s')
        df['boost_accrual_start_time_days'] = (df['boost_accrual_start_time'] / 1000 / 3600 / 24)
        df['boost_accrual_start_time_hours'] = df['boost_accrual_start_time'] / 1000 / 3600
        df['boost_accrual_start_time_days_int'] = df['boost_accrual_start_time_days'].apply(int)
        self.amps = df

In [293]:
class AMPSChart:
    
    def __init__(self):
        self.cols_dict = {
            'user_xprism': 'Amount of xPRISM',
            'boost_accrual_start_time_days_int': 'Number of days pledged for',
            'index': 'Number of users',
            'boost_accrual_start_time_days': 'Current number of days pledged for',
            'user_yluna': 'Amount of yLUNA',
            'addr': 'User address',
            'current_daily_rewards': 'Current daily rewards (PRISM)',
            'n_addr': 'Number of users',
            'boost_apr': 'Boost APR (%)'
        }

    def time_xprism_yluna(self, df):
        cols_dict = self.cols_dict
        df['url'] = 'https://finder.extraterrestrial.money/mainnet/address/'+df['addr']
        if(len(df)>5000):
            df = df.sample(n=5000, random_state=1)
        else:
            df = df
        df2 = df.rename(columns=cols_dict)
        chart =alt.Chart(df2).mark_point(opacity=1, filled=True).encode(
            y=alt.Y(cols_dict['user_xprism']+":Q"),
            x=alt.X(cols_dict['boost_accrual_start_time_days']+":Q"),
            href='url:N',
            color=alt.Color(cols_dict['user_yluna'],
                scale=alt.Scale(scheme='lightgreyteal')),
            tooltip=[cols_dict['addr'],cols_dict['user_xprism'],cols_dict['user_yluna'],
                     cols_dict['boost_accrual_start_time_days']]
        ).configure_view(strokeOpacity=0).interactive()
        return chart.properties(width=600)
    
    def users_daily_rewards(self, df):
        cols_dict = self.cols_dict
        df2 = df.copy()
        df2['current_daily_rewards'] = df2['current_daily_rewards'].apply(lambda x: 1 if int(x/10)*10 == 0 else int(x/10)*10)
        df2['n_addr'] = df2['addr']
        df2 = df2.groupby('current_daily_rewards').n_addr.count().reset_index()
        df2 = df2.rename(columns=cols_dict)
        chart = alt.Chart(df2).mark_bar().encode(
            x=alt.X(cols_dict['current_daily_rewards']+':N', \
                    axis=alt.Axis(tickCount=10, labelAngle=-90, tickBand = 'center')),
            y=cols_dict['n_addr']+':Q',
            tooltip=[cols_dict['current_daily_rewards'],cols_dict['n_addr']+':Q']
        ).configure_mark(
            color='#DAFD91'
        ).configure_view(strokeOpacity=0)
        return chart.properties(width=600)
    
    def users_boost_apr(self, df):
        cols_dict = self.cols_dict
        df2 = df.copy().fillna(0)
        df2['boost_apr'] = df2['boost_apr'].apply(lambda x: 0 if int(x/10)*10 == 0 else int(x/10)*10)
        df2['n_addr'] = df2['addr']
        df2 = df2.groupby('boost_apr').n_addr.count().reset_index()
        df2 = df2.rename(columns=cols_dict)
        chart = alt.Chart(df2).mark_bar().encode(
            x=alt.X(cols_dict['boost_apr']+':N', \
                    axis=alt.Axis(tickCount=10, labelAngle=-90, tickBand = 'center')),
            y=cols_dict['n_addr']+':Q',
            tooltip=[cols_dict['boost_apr'],cols_dict['n_addr']+':Q']
        ).configure_mark(
            color='#ccf4ed'
        ).configure_view(strokeOpacity=0)
        return chart.properties(width=600)
    
    def users_days_pledged(self,df):
        cols_dict = self.cols_dict
        df2 = df.boost_accrual_start_time_days.apply(int).value_counts().reset_index()
        df2 = df2.rename(columns=cols_dict)
        chart = alt.Chart(df2).mark_bar().encode(
            x=alt.X(cols_dict['index']+':N', \
                    scale=alt.Scale(domain=list(range(df2[cols_dict['index']].max()+1))),\
                    axis=alt.Axis(tickCount=10, labelAngle=0, tickBand = 'center')),
            y=cols_dict['boost_accrual_start_time_days']+":Q",
            tooltip=[cols_dict['index'],cols_dict['boost_accrual_start_time_days']+":Q"]
        ).configure_mark(
            color='#DAFD91'
        ).configure_view(strokeOpacity=0)
        return chart.properties(width=600)
    
    def xprisms_days_pledged(self,df):
        cols_dict = self.cols_dict
        df2 = df.groupby('boost_accrual_start_time_days_int').user_xprism.sum().reset_index()
        df2 = df2.rename(columns=cols_dict)
        chart = alt.Chart(df2).mark_bar().encode(
            x=alt.X(cols_dict['boost_accrual_start_time_days_int']+':N', \
                    scale=alt.Scale(domain=list(range(df2[cols_dict['boost_accrual_start_time_days_int']].max()+1))),\
                    axis=alt.Axis(tickCount=10, labelAngle=0, tickBand = 'center')),
            y=cols_dict['user_xprism']+':Q',
            tooltip=[cols_dict['boost_accrual_start_time_days_int'],cols_dict['user_xprism']+':Q']
        ).configure_mark(
            color='#ccf4ed'
        ).configure_view(strokeOpacity=0)
        return chart.properties(width=600)

In [294]:
dp = AMPSDataProvider()
dp.load()
dp.parse()

File not found: ../data/amps_20220322_1.csv
File found: ../data/amps_20220321_1.csv


## Query

In [295]:
cp = AMPSChart()

In [296]:
cp.time_xprism_yluna(dp.amps)

alt.Chart(...)

In [297]:
cp.users_daily_rewards(dp.amps)

alt.Chart(...)

In [298]:
cp.users_boost_apr(dp.amps)

alt.Chart(...)

In [299]:
cp.users_days_pledged(dp.amps)

alt.Chart(...)

In [300]:
cp.xprisms_days_pledged(dp.amps)

alt.Chart(...)